In [0]:
import pyspark
from pyspark.sql import SparkSession

In [0]:
spark=SparkSession.builder.appName('log_reg_titanic').getOrCreate()

In [0]:
df=spark.read.csv('/FileStore/tables/titanic.csv',inferSchema=True, header=True)

In [0]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
PassengerId|Survived|Pclass| Name| Sex| Age|SibSp|Parch| Ticket| Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
 1| 0| 3|Braund, Mr. Owen ...| male|22.0| 1| 0| A/5 21171| 7.25| null| S|
 2| 1| 1|Cumings, Mrs. Joh...|female|38.0| 1| 0| PC 17599|71.2833| C85| C|
 3| 1| 3|Heikkinen, Miss. ...|female|26.0| 0| 0|STON/O2. 3101282| 7.925| null| S|
 4| 1| 1|Futrelle, Mrs. Ja...|female|35.0| 1| 0| 113803| 53.1| C123| S|
 5| 0| 3|Allen, Mr. Willia...| male|35.0| 0| 0| 373450| 8.05| null| S|
 6| 0| 3| Moran, Mr. James| male|null| 0| 0| 330877| 8.4583| null| Q|
 7| 0| 1|McCarthy, Mr. Tim...| male|54.0| 0| 0| 17463|51.8625| E46| S|
 8| 0| 3|Palsson, Master. ...| male| 2.0| 3| 1| 349909| 21.075| null| S|
 9| 1| 3|Johnson, Mrs. Osc...|female|27.0| 0| 2| 347742|11.1333| null| S|
 10| 1| 2|Nasser, Mrs. Nich...|female|14.0| 1| 0| 237736|30.0708| null| C|
 11| 1| 3|Sandstrom, Miss. ...|female| 4.0| 1| 1| PP 9549| 16.7| G6| S|
 12| 1| 1|Bonnell, Miss. El...|female|58.0| 0| 0| 113783| 26.55| C103| S|
 13| 0| 3|Saundercock, Mr. ...| male|20.0| 0| 0| A/5. 2151| 8.05| null| S|
 14| 0| 3|Andersson, Mr. An...| male|39.0| 1| 5| 347082| 31.275| null| S|
 15| 0| 3|Vestrom, Miss. Hu...|female|14.0| 0| 0| 350406| 7.8542| null| S|
 16| 1| 2|Hewlett, Mrs. (Ma...|female|55.0| 0| 0| 248706| 16.0| null| S|
 17| 0| 3|Rice, Master. Eugene| male| 2.0| 4| 1| 382652| 29.125| null| Q|
 18| 1| 2|Williams, Mr. Cha...| male|null| 0| 0| 244373| 13.0| null| S|
 19| 0| 3|Vander Planke, Mr...|female|31.0| 1| 0| 345763| 18.0| null| S|
 20| 1| 3|Masselmani, Mrs. ...|female|null| 0| 0| 2649| 7.225| null| C|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 20 rows

In [0]:
df.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [0]:
df.columns

Out[17]: ['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [0]:
# Fetching the essential fields to make the feature column
my_cols=df.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [0]:
# Getting rid of null values
my_final_data=my_cols.na.drop()

In [0]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, StringIndexer, OneHotEncoder

In [0]:
# assigning number to every category of the column sex (technically indexing)
gender_indexer=StringIndexer(inputCol='Sex',outputCol='SexIndex')
# For example - indexing
# X Y Z
# 1 2 3
# Example - ONE HOT ENCODE
# KEY X Y Z
# Case : Y
# [0, 1, 0]
gender_encoder=OneHotEncoder(inputCol='SexIndex',outputCol='SexVector')

In [0]:
embarked_indexer=StringIndexer(inputCol='Embarked',outputCol='EmbarkedIndex')
embarked_encoder=OneHotEncoder(inputCol='EmbarkedIndex',outputCol='EmbarkedVector')

In [0]:
assembler=VectorAssembler(inputCols=['Pclass','SexVector','EmbarkedVector','Age','SibSp','Parch','Fare'],
                         outputCol='features')

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
from pyspark.ml import Pipeline

In [0]:
log_reg_titanic=LogisticRegression(featuresCol='features',labelCol='Survived')

In [0]:
pipeline_titanic=Pipeline(stages=[gender_indexer,gender_encoder,embarked_indexer,embarked_encoder,assembler,log_reg_titanic])

In [0]:
train_data, test_data=my_final_data.randomSplit([0.7,0.3])

In [0]:
fit_model=pipeline_titanic.fit(train_data)

In [0]:
results=fit_model.transform(test_data)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
my_evaluation=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [0]:
results.select('Survived','prediction').show()

+--------+----------+
Survived|prediction|
+--------+----------+
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+--------+----------+
only showing top 20 rows

In [0]:
# Area Under Curve
AUC=my_evaluation.evaluate(results)

In [0]:
AUC

Out[45]: 0.7866479925303455